<h1> Classifying class 1/class 2 of GTSRB

In [31]:
import cv2
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.svm import SVC

In [18]:
#read csv
df = pd.read_csv('Train.csv')
df.head(10)

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png
5,31,27,6,5,26,22,20,Train/20/00020_00000_00005.png
6,31,28,6,6,26,23,20,Train/20/00020_00000_00006.png
7,31,28,6,6,26,23,20,Train/20/00020_00000_00007.png
8,31,29,5,6,26,24,20,Train/20/00020_00000_00008.png
9,34,32,6,6,29,26,20,Train/20/00020_00000_00009.png


In [6]:
paths1 = df[(df['ClassId'] ==1)]['Path'].values
paths2 = df[(df['ClassId'] ==2)]['Path'].values

class1_arrs = list()
for path in paths1:
    image = cv2.imread(path) #list of list of rgb values
    image_from_array = Image.fromarray(image, 'RGB')
    #resize images to 30x30
    resized_image = image_from_array.resize((30,30))
    #convert to an array
    image_arr = np.array(resized_image.getdata(),
                    np.uint8).reshape(resized_image.size[1], resized_image.size[0], 3)
    class1_arrs.append(image_arr)
    
    
class2_arrs = list()    
for path in paths2:
    image = cv2.imread(path)
    image_from_array = Image.fromarray(image, 'RGB')
    resized_image = image_from_array.resize((30,30))
    image_arr = np.array(resized_image.getdata(),
                         np.uint8).reshape(resized_image.size[1], resized_image.size[0], 3)
    class2_arrs.append(image_arr)


In [7]:
#all the images
X = class1_arrs + class2_arrs
#list with class 1 classified as 0, class2 classified as 1
y = len(class1_arrs)*[0] + len(class2_arrs)*[1]

In [8]:
len(X) == len(y)

True

In [9]:
#flatten images

c = 0
while c < len(X):
    X[c] = np.array(X[c]).flatten()
    c +=1   

print(X[0])

[181 179 180 ... 141 140 151]


In [10]:
len(X) == len(y)

True

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

3576 894 3576 894


In [13]:
X_train[:10]

[array([21, 20, 21, ..., 23, 27, 26], dtype=uint8),
 array([ 62,  63,  62, ..., 184, 177, 178], dtype=uint8),
 array([17, 22, 25, ..., 18, 26, 29], dtype=uint8),
 array([31, 25, 46, ..., 95, 67, 55], dtype=uint8),
 array([44, 45, 46, ..., 26, 28, 37], dtype=uint8),
 array([126, 179, 176, ...,  46,  69,  71], dtype=uint8),
 array([147, 148, 151, ..., 135, 139, 133], dtype=uint8),
 array([18, 18, 19, ..., 44, 72, 64], dtype=uint8),
 array([252, 255, 255, ..., 242, 252, 193], dtype=uint8),
 array([ 65, 116, 147, ...,  63,  64,  59], dtype=uint8)]

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train[:10])

[[-0.90554976 -1.01221048 -1.05930886 ... -0.78323668 -0.79287721
  -0.84039854]
 [-0.37008663 -0.44525344 -0.5273924  ...  1.70830996  1.52058165
   1.46322991]
 [-0.95779006 -0.98584038 -1.00741457 ... -0.8606139  -0.80830027
  -0.79493219]
 ...
 [-0.94472999 -1.03858057 -1.085256   ... -0.45825233 -0.09883955
  -0.26449143]
 [ 2.11132788  2.08627564  1.97650703 ...  2.60588577  2.67731108
   1.69056167]
 [-0.3309064   0.25355406  0.57536124 ... -0.16421888 -0.22222403
  -0.34026868]]


<h2> Log Reg Classifier

In [15]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
lr.score(X_train, y_train)

1.0

In [22]:
lr.score(X_test, y_test)

0.9664429530201343

<h2> Random Forest Classifier

In [26]:
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
rfc.score(X_train, y_train)

1.0

In [28]:
rfc.score(X_test, y_test)

0.9765100671140939

In [29]:
print(classification_report(y_test, pred_vals))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       452
           1       0.97      0.98      0.98       442

    accuracy                           0.98       894
   macro avg       0.98      0.98      0.98       894
weighted avg       0.98      0.98      0.98       894



In [192]:
print(confusion_matrix(y_test, pred_vals))

[[440  12]
 [  8 434]]


<h2> SVM

In [32]:
svm = SVC()
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [33]:
svm.score(X_train, y_train)

0.9661633109619687

In [34]:
svm.score(X_test, y_test)

0.9541387024608501